In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split

from torchvision import models
import torchvision.utils

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# helper function
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize=(5, 15))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis("off")
    plt.show()

In [3]:
# Use GPU if available, else use CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use metal for mac m1
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print(
            "MPS not available because the current PyTorch install was not "
            "built with MPS enabled."
        )
    else:
        print(
            "MPS not available because the current MacOS version is not 12.3+ "
            "and/or you do not have an MPS-enabled device on this machine."
        )

else:
    device = torch.device("mps")
    print(device)

mps


<h1> Prepare Data</h1>

In [4]:
import dataset

# seed rng to reproduce
torch.manual_seed(42)
batch_size=128

dataset = dataset.create_dataset()
train_size = int(0.8 * len(dataset))  # 80% of data for training
test_size = len(dataset) - train_size  # Remaining 20% for testing

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print(len(dataset))
# print(len(train_loader))
# for batch_number, (inputs, labels) in enumerate(train_loader):
#     print(batch_number, inputs, labels)

100%|██████████| 4/4 [00:00<00:00, 737.17it/s]
/Users/yufangoh/Work/deepLearning_AlzheimerDetection/dataset.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": ad_labels.LABELS_MAP})


Use pretrained inception

In [5]:
model = models.inception_v3(pretrained=True)
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

# model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 10), nn.Linear(10, 2))
model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 4))

model = model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=0.001
)


num_epochs = 10

for epoch in range(num_epochs):
    total_batch = len(train_dataset) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = batch_images.to(device)
        Y = batch_labels.to(device)

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i + 1) % 5 == 0:
            print(
                "Epoch [%d/%d], lter [%d/%d] Loss: %.4f"
                % (epoch + 1, num_epochs, i + 1, total_batch, cost.item())
            )

/Users/yufangoh/Work/deepLearning_AlzheimerDetection/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yufangoh/Work/deepLearning_AlzheimerDetection/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], lter [5/40] Loss: 1.0150
Epoch [1/10], lter [10/40] Loss: 1.0098
Epoch [1/10], lter [15/40] Loss: 1.0294
Epoch [1/10], lter [20/40] Loss: 0.9740
Epoch [1/10], lter [25/40] Loss: 1.0313
Epoch [1/10], lter [30/40] Loss: 0.9281
Epoch [1/10], lter [35/40] Loss: 0.9641
Epoch [1/10], lter [40/40] Loss: 1.0180
Epoch [2/10], lter [5/40] Loss: 1.0530
Epoch [2/10], lter [10/40] Loss: 1.0458
Epoch [2/10], lter [15/40] Loss: 0.9038
Epoch [2/10], lter [20/40] Loss: 1.0314
Epoch [2/10], lter [25/40] Loss: 0.9869
Epoch [2/10], lter [30/40] Loss: 1.0143
Epoch [2/10], lter [35/40] Loss: 0.8336
Epoch [2/10], lter [40/40] Loss: 0.8067
Epoch [3/10], lter [5/40] Loss: 0.9487
Epoch [3/10], lter [10/40] Loss: 0.9945
Epoch [3/10], lter [15/40] Loss: 0.8946
Epoch [3/10], lter [20/40] Loss: 0.9471
Epoch [3/10], lter [25/40] Loss: 0.9219
Epoch [3/10], lter [30/40] Loss: 0.9886
Epoch [3/10], lter [35/40] Loss: 0.9706
Epoch [3/10], lter [40/40] Loss: 0.8935
Epoch [4/10], lter [5/40] Loss: 0.9622
Epoc

In [7]:
import ad_labels
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    images = images.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.to(device)).sum()

print("Accuracy of test images: %f %%" % (100 * float(correct) / total))

classes = ad_labels.LABELS_LIST

images, labels = iter(test_loader).next()

outputs = model(images.to(device))

_, predicted = torch.max(outputs.data, 1)

print("Predicted: ", " ".join("%5s" % classes[predicted[j]] for j in range(5)))

title = " ".join("%5s" % classes[labels[j]] for j in range(5))
imshow(torchvision.utils.make_grid(images, normalize=True), title)

Accuracy of test images: 60.312500 %


AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'